In [1]:
import copy
from difflib import SequenceMatcher
import difflib
import os
import pandas as pd
import pyperclip
from tqdm import tqdm

In [2]:
formulario = pd.read_excel('Formularios contacto.xlsx')
formulario.tail(3)

,RUT,Nombre,Apellido,Email,Teléfono,Empresa o institución,Región,Número de usuarios,Tipo de usuario,Selecciona tu representante de ventas,Comentarios,Creación
72,NaN,Luis,San Martín.,lusanmartin@gmail.com,'+56 945309626,Constructora PZ Patricio Zamora Vergara,Valparaíso,536 personas,Habitantes temporales,NaN,Alejandra Ganter\n\ncontamos con anteproyecto ...,2020-10-14 12:47:13
73,NaN,ELADIO,MUÑOZ,gerencia@himax.cl,996406036,EMPRESA,Aysén,20,Habitantes permanentes,NaN,"Estimada Alejandra, agradeceré cotizar planta ...",2020-09-30 18:08:45
74,NaN,Inti,Peraldi,inti@ayma.cl,'+56932659710,AYMA Arquitectura y Medio Ambiente Ltda.,Metropolitana,40,Habitantes permanentes,NaN,Buenos días: Les agradecería pudieran cotizar ...,2020-09-21 10:30:40


In [3]:
comunas_de_chile = pd.read_html('https://es.wikipedia.org/wiki/Anexo:Comunas_de_Chile')[0]
comunas_de_chile.drop(columns=['Unnamed: 2'], inplace=True)
comunas_de_chile.tail(3)

,CUT (Código Único Territorial),Nombre,Provincia,Región,Superficie (km²),Población 2020,Densidad (hab./km²),IDH 2005,IDH 2005.1,Latitud,Longitud
343,13603,Isla de Maipo,Talagante,Metropolitana de Santiago,189,40.171,212.5,0.724,Alto,"-33°45'14.4""","-70°53'9.6"""
344,13604,Padre Hurtado,Talagante,Metropolitana de Santiago,80.8,74.188,915.9,0.728,Alto,"-33°34'33.6""","-70°47'60"""
345,13605,Peñaflor,Talagante,Metropolitana de Santiago,69,101.058,"1.464,6",0.753,Alto,"-33°36'22.6""","-70°52'35.48"""


In [4]:
def get_match(df, campo, busqueda):
    marcas = df[campo].tolist()
    match = {}

    for marca in marcas:
        s = SequenceMatcher(None, busqueda, marca)
        match[s.ratio()] = marca

    maximo = max(match.keys())
    df[df[campo] == match[maximo]]

    return {'matches': match, 'maximo': maximo}


def get_geography(comunas_de_chile, possible_ids, campo, region):
    relevant_cols = ['Nombre', 'Provincia', 'Región']
    possible_results = []
    
    # Regiones de Rutificador
    for i in possible_ids.index:
        location = possible_ids.loc[i, 'Ciudad/Comuna']
        dict_matches = get_match(comunas_de_chile, campo, location)
        maximo = dict_matches['maximo']

        if maximo > 0.8:
            possible_results.append(dict_matches['matches'][maximo])

    # Filtro por Alternativas de Rutificador y Región predefinida
    geography = comunas_de_chile[(comunas_de_chile[campo].isin(possible_results)) &
                                 (comunas_de_chile['Región'] == region)][relevant_cols].copy(deep=True)

    # Transformación del resultado de comunas e identificación
    geography.rename(columns={'Nombre': 'Comuna'}, inplace=True)
    geography.reset_index(inplace=True, drop=True)
    
    return geography


def get_identification(geography, possible_ids):
    cities = geography['Comuna'].to_list()
    possibilities = possible_ids[possible_ids['Ciudad/Comuna'].isin(cities)].copy(deep=True)

    for i in possibilities.index:
        formal_name = possibilities.loc[i, 'Nombre']
        split_string = formal_name.split(' ')

        # Creating new name based on a street constructed name
        street_name = f"{' '.join(split_string[2:])} {' '.join(split_string[:2])}"
        possibilities.loc[i, 'Nombre'] = street_name

    dict_matches = get_match(possibilities, campo='Nombre', busqueda=formal_name)
    matching_id = dict_matches['matches'][dict_matches['maximo']]

    identification = possibilities[possibilities['Nombre'] == street_name].copy(deep=True)
    identification['id_certainty'] = dict_matches['maximo']
    identification.reset_index(inplace=True, drop=True)

    return identification

In [7]:
# Actual Excecution
if 'last_index' not in locals():
    last_index, list_rows = 0, []

for i in tqdm(formulario.index):

    # Controls       
    if i < last_index:
        continue

    contact = formulario[formulario.index == i].copy(deep=True)
    contact.reset_index(inplace=True, drop=True)

    # Obtenemos el nombre y la región del contacto
    name = f"{contact.loc[0, 'Nombre']} {contact.loc[0, 'Apellido']}"
    forms_region = contact['Región']
    pyperclip.copy(name)

    # Obtenemos la tabla de identificación de Rutificador, además de la región según Wikipedia
    dict_matches = get_match(comunas_de_chile, campo='Región', busqueda=forms_region)
    match_region = dict_matches['matches'][dict_matches['maximo']]

    # print(f'\n\nExtrayendo información de {name} de la región de {match_region}')

    # Clipboard table or command
    clipboard_table = input('Introduzca la tabla copiada: ')
    if clipboard_table == 'continue':
        continue

    # Añadimos la región y la provincia en base a la comuna
    possible_ids = pd.read_clipboard(sep='\t')
    geography = get_geography(comunas_de_chile, possible_ids, campo='Nombre',
                              region=match_region)

    identification = get_identification(geography, possible_ids)

    # Multiindex and merging of dataframes
    dict_multiindex = {'Form': contact, 'Geography': geography, 'Identification': identification}
    df_merged = pd.DataFrame({('Base', 'Blank'): None}, index=[0])

    for key, series in dict_multiindex.items():
        new_series = series.copy(deep=True)

        new_series.columns = pd.MultiIndex.from_tuples((key, col) for col in series.keys())
        df_merged = pd.merge(df_merged, new_series, left_index=True, right_index=True)

    df_merged.drop(columns=('Base', 'Blank'), inplace=True)

    list_rows.append(df_merged)
    last_index = i

    # display(df_merged)

 16%|█▌        | 12/75 [00:42<03:43,  3.54s/it]


KeyError: 'Ciudad/Comuna'

In [8]:
last_index

12